# Using a genetic algorithm to train an agent in the Game Of Life

Types of agent:
    - Glider
    - 
    

    
Method:
    - Define a fitness function for agent
    - Generate a certain number of dna
    - Test the fitness of all the dna
    - Pick the best dna from the pool, mutate them
    - Repeat until you have a satisfactory agent that can perform the task specified

In [5]:
import cv2 as cv
import random
import numpy as np
from dna import *
from gol import *

Display code

In [2]:
def animate_board(gol, frame_rate = 10):
    rootWindow = "  --  GAME OF LIFE!!  --  "
    cv.namedWindow(rootWindow)

#     np.random.seed(0)

    scale_factor = np.int32(1000 / gol.dim[0])
    
    
    while True:
        key = cv.waitKey(np.int32(1000/frame_rate))

        img = gol.board.copy()
        img = img * 255
        img = img.astype(np.uint8)
        
        for i in range(int(np.log2(scale_factor))):

            img = np.repeat(np.repeat(img, 2, axis=0), 2, axis=1)

        cv.imshow(rootWindow, img)

        if not gol.next_frame() or key == 32:
            break
    cv.destroyAllWindows()

In [8]:
cv.destroyAllWindows()


Fitness Functioons

In [ ]:
def glider_fitness(agent):
    
    score = 0
    ## Constants
    DIM = (50, 50)
    STARTING_POS = (10, 10)
    
    ## Set up the board
    gol_board = GOL(DIM)
    gol_board.add_agent(STARTING_POS, agent)
    
    
    while 
    

Genetic Algorithm

In [7]:
board_dimention = (50, 50)

scale_factor = 20

# Create the game of life baord
gol_board = GOL(board_dimention)

# Create a random agent
agent = DNA((5, 5))
# Define the position we would like to place the agent
agent_pos = (5, 5)


# gol_board.add_agent(agent_pos, agent)

mat = np.array([[0, 1, 0], [0, 0, 1], [1, 1, 1]])


gol_board.modify((5, 5), mat)

animate_board(gol_board, frame_rate = 30)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

Distributive code

In [ ]:
# pip install -q git+https://github.com/Distributive-Network/Bifrost


# DCP MODULE
from bifrost import dcp

##################
# INPUT SET
##################
b_set = range(12)
##################
# WORK FUNCTION
##################
"Compute left Riemann sum of exp(-x^2) from 0 to b with N subintervals."
def riemann_sum(b,N=100000):

  # packages required inside the work function must be imported inside the work function
  import numpy as np

  x = np.linspace(0,b,N+1)
  x_left_endpoints = x[:-1]
  Delta_x = b/N
  I = Delta_x * np.sum(np.exp(-x_left_endpoints**2))

  return I

##################
# JOB & CONFIG
##################
job = dcp.compute_for(b_set, riemann_sum)
job.requires('numpy')
job.compute_groups = [{'joinKey': 'demo', 'joinSecret': 'dcp'}]
job.public['name'] = "Numpy Riemann Sums Job, on DCP!"

##################
# EXEC
##################
results = job.exec()


print(results)



In [11]:
matrix = np.array([[1, 2, 3], [4, 5, 6]])

print(matrix.shape)

(2, 3)
